T5-Base

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
!pip install tqdm

In [ ]:
!pip install evaluate
!pip install bert-score
!pip install rouge-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.wh

In [ ]:
!pip install textstat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 40.9 MB/s eta 0:00:00


In [ ]:
import re
import random
import pandas as pd
import torch
from transformers import T5TokenizerFast, T5ForConditionalGeneration
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
from rouge_score import rouge_scorer
import bert_score
from bert_score import score as bert_score_fn
import logging

nltk.download('punkt')  # for sentence tokenization
from google.colab import drive
drive.mount('/content/drive')

logging.getLogger().setLevel(logging.WARNING)
logging.getLogger("bert_score").setLevel(logging.ERROR)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Mounted at /content/drive


In [ ]:
import re
import random
import pandas as pd
import torch
from transformers import T5TokenizerFast, T5ForConditionalGeneration
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
# from rouge_score import rouge_scorer
import bert_score
# from bert_score import score as bert_score_fn
from nltk.translate.meteor_score import meteor_score
import evaluate
import logging

nltk.download('punkt')  # for sentence tokenization
from google.colab import drive
drive.mount('/content/drive')

logging.getLogger().setLevel(logging.WARNING)
logging.getLogger("bert_score").setLevel(logging.ERROR)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import math
import torch, evaluate
import numpy as np
from bert_score import score
from datasets import load_metric
from evaluate import load as evaluate_load
from bert_score import score as bert_score_fn
import os
import string
import textstat
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

**split** in blocks

T5

In [ ]:
class MLMAugmentor:
    def __init__(self, notes_path, annotations_path, small_size=10, random_seed=42, masking_ratio=0.3, k=3, min_lines=3):
        self.notes_path = notes_path
        self.annotations_path = annotations_path
        self.small_size = small_size
        self.random_seed = random_seed
        self.masking_ratio = masking_ratio
        self.k = k
        self.min_lines = min_lines
        self.invalid_masks = 0
        self.total_masks = 0
        random.seed(self.random_seed)  # for reproducibility
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.tokenizer = T5TokenizerFast.from_pretrained('t5-base')
        self.model = T5ForConditionalGeneration.from_pretrained('t5-base').to(self.device)
        # self.scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
        self.load_and_merge_datasets()
        self.rouge = evaluate_load('rouge')

    def load_and_merge_datasets(self):
        try:
            notes_df = pd.read_csv(self.notes_path)
            annotations_df = pd.read_csv(self.annotations_path)
            self.merged_df = pd.merge(notes_df, annotations_df, on='note_id', how='left')
        except Exception as e:
            print(f"An error occurred: {e}")

    def split_into_sentences(self, text, min_lines=3):
        # Splitting text into sentence
        sentences = sent_tokenize(text)
        processed_sentences = []
        current_block = []
        num_lines = 0


        for sentence in sentences:
            current_block.append(sentence)
            num_lines += sentence.count('\n') + 1

            if num_lines >= min_lines:
                processed_sentences.append(" ".join(current_block))
                current_block = []
                num_lines = 0

        # Add the last block if it's not empty
        if current_block:
            processed_sentences.append(" ".join(current_block))

        return processed_sentences

    def process_text(self, note_id, text, annotations):
        sentences = self.split_into_sentences(text, self.min_lines)
        start_index = 0
        for i, sentence in enumerate(sentences[:self.k]):
            sentence_start = text.find(sentence)
            sentence_end = sentence_start + len(sentence)
            entities = [(text[s:e], s, e, cid) for s, e, cid in annotations if s >= sentence_start and e <= sentence_end]
            masked_sentence, structure = self.mask_text(sentence, entities, sentence_start, sentence_end)
            regenerated_sentence = self.regenerate_text(masked_sentence)

            # Replace <extra_id_x> with <mask> for printing
            display_masked_sentence = re.sub(r'<extra_id_\d+>', '<mask>', masked_sentence)
            print(f"\n\033[1mNote ID: {note_id}, Sentence {i+1}\033[0m")
            print("\n\033[1mOriginal Sentence:\033[0m\n", sentence)
            print("\n\033[1mEntities:\033[0m\n", entities)
            print("\n\033[1mStructure:\033[0m\n", structure)
            print("\n\033[1mMasked Sentence:\033[0m\n", display_masked_sentence)
            print("\n\033[1mGenerated Text:\033[0m\n", regenerated_sentence)

            # # Compute ROUGE-L score
            # rouge_scores = self.scorer.score(sentence, regenerated_sentence)['rougeL']
            # print("\n\033[1mROUGE-L Score:\033[0m")
            # print(f"Precision: {rouge_scores.precision:.4f}, Recall: {rouge_scores.recall:.4f}, F1: {rouge_scores.fmeasure:.4f}")

            # # Compute BERTScore using 'bert-base-uncased' model
            # P, R, F1 = bert_score_fn([regenerated_sentence], [sentence], lang="en", model_type="bert-base-uncased", rescale_with_baseline=True)
            # # P, R, F1 = bert_score_fn([regenerated_sentence], [sentence], lang="en", model_type="bert-base-uncased", rescale_with_baseline=True, show_progress_bar=False)
            # print("\n\033[1mBERTScore:\033[0m")
            # print(f"Precision: {P.mean().item():.4f}, Recall: {R.mean().item():.4f}, F1: {F1.mean().item():.4f}")
            rouge_results = self.rouge.compute(predictions=[regenerated_sentence], references=[sentence])
            P, R, F1 = bert_score_fn([regenerated_sentence], [sentence], lang="en", rescale_with_baseline=True)

            rouge_1 = rouge_results['rouge1'] * 100
            rouge_2 = rouge_results['rouge2'] * 100
            rouge_L = rouge_results['rougeL'] * 100
            bertscore_f1 = F1.mean().cpu().item()
            flesch_reading_ease = textstat.flesch_reading_ease(regenerated_sentence)

            # METEOR score
            tokenized_sentence = word_tokenize(sentence)
            tokenized_regeneration = word_tokenize(regenerated_sentence)
            meteor = meteor_score([tokenized_sentence], tokenized_regeneration)

            flesch_reading_ease_ori = textstat.flesch_reading_ease(sentence)
            flesch_reading_ease_mask = textstat.flesch_reading_ease(masked_sentence)
            rouge_results_mask = self.rouge.compute(predictions=[masked_sentence], references=[sentence])
            rouge_1_mask = rouge_results_mask['rouge1'] * 100
            rouge_2_mask = rouge_results_mask['rouge2'] * 100
            rouge_L_mask = rouge_results_mask['rougeL'] * 100

            P_Mask, R_Mask, F1_Mask = bert_score_fn([masked_sentence], [sentence], lang="en", rescale_with_baseline=True)
            bertscore_f1_mask = F1_Mask.mean().cpu().item()

            # METEOR score
            tokenized_masked_sentence = word_tokenize(masked_sentence)
            meteor_mask = meteor_score([tokenized_sentence], tokenized_masked_sentence)

            print(f"\n\033[1mMetrics for Note ID: {note_id}, Sentence {i+1}\033[0m:")
            print("{:<25} {:<15} {:<15}".format("Metric", "Evaluation", "Baseline"))
            print("-" * 80)
            print("{:<25} {:<15.2f} {:<15.2f}".format("ROUGE-1", rouge_1, rouge_1_mask))
            print("{:<25} {:<15.2f} {:<15.2f}".format("ROUGE-2", rouge_2, rouge_2_mask))
            print("{:<25} {:<15.2f} {:<15.2f}".format("ROUGE-L", rouge_L, rouge_L_mask))
            print("{:<25} {:<15.2f} {:<15.2f}".format("BERTScore F1", bertscore_f1, bertscore_f1_mask))
            print("{:<25} {:<15.2f} {:<15.2f}".format("METEOR", meteor, meteor_mask))
            print("{:<25} {:<15.2f} {:<15}".format("Flesch Reading Ease", flesch_reading_ease, f"Ori: {flesch_reading_ease_ori:.2f}, Mask: {flesch_reading_ease_mask:.2f}"))


            print("="*80)

    def mask_text(self, sentence, entities, sentence_start, sentence_end):
        tokens = word_tokenize(sentence)
        original_tokens = tokens[:]  # Copy of original tokens
        mask = [True] * len(tokens)  # Initialize mask list, assuming all tokens can be masked

        # Generate character offsets for each token using word_tokenize positions
        token_offsets = []
        current_position = 0
        for token in tokens:
            start_position = sentence.find(token, current_position)
            end_position = start_position + len(token)
            token_offsets.append((start_position, end_position))
            current_position = end_position

        structure_texts = []
        allowed_lower = {'a', 'the', 'of', 'in', 'on', 'at', 'or', 'by', 'for', 'with', 'about', 'as', 'an', 'to'}
        termination_punctuation = {'.', '_', '___', '!', '?', ':'}
        # non_termination_pattern = re.compile(r'[^a-zA-Z\.!_?___]')
        non_termination_pattern = re.compile(r'[^.!:_?___]')

        i = 0
        while i < len(tokens):
            if tokens[i].endswith(':'):
                # Start checking backwards from the colon
                title_start = i - 1
                while title_start >= 0:
                    current_token = tokens[title_start]
                    if current_token in termination_punctuation or (current_token.islower() and current_token not in allowed_lower):
                        break  # Stop if a termination punctuation or a non-allowed lowercase word is encountered
                    title_start -= 1
                title_start += 1  # Adjust to include the first valid word after the break point

                # Capture the title segment if valid
                if title_start < i:
                    title_segment = tokens[title_start:i + 1]  # Include the colon in the title
                    structure_text = ' '.join(title_segment)
                    if not structure_texts or (structure_texts and structure_text not in structure_texts[-1]):
                        structure_texts.append(structure_text)
                        # Mask the title segment
                        for j in range(title_start, i + 1):
                            mask[j] = False
                i = i + 1  # Ensure to move past the last processed colon
            else:
                i += 1

        # Protect entities based on their positions and non-alphabetic tokens
        special_pattern = re.compile(r'\w\s*[^\w\s]\s*\d')
        for idx, token in enumerate(tokens):
            if not token.isalpha() or token.isupper() or re.search(r'\d', token) or special_pattern.search(token):  # Check if the token is purely alphabetic
                mask[idx] = False

        # only mask entities (identify borders accuratly)
        for _, ent_start, ent_end, _ in entities:
            for idx, (token_start, token_end) in enumerate(token_offsets):
                if (token_start + sentence_start >= ent_start and token_start + sentence_start < ent_end) or \
                  (token_end + sentence_start > ent_start and token_end + sentence_start <= ent_end) or \
                  (token_start + sentence_start <= ent_start and token_end + sentence_start >= ent_end):
                    mask[idx] = False

        for match in re.finditer(r'\w\s*[^\w\s]\s*\d', sentence):
            match_start, match_end = match.span()
            for idx, (token_start, token_end) in enumerate(token_offsets):
                if token_start >= match_start and token_end <= match_end:
                    mask[idx] = False
                    break

        # Calculate the indices of maskable tokens
        maskable_indices = [idx for idx, m in enumerate(mask) if m]
        if self.masking_ratio == 1.0:
            # If mask_ratio is 1.0, mask all maskable tokens
            mask_indices = maskable_indices
        else:
            num_to_mask = int(len(maskable_indices) * self.masking_ratio)
            mask_indices = random.sample(maskable_indices, num_to_mask) if maskable_indices else []

        mask_id = 0  # Initialize mask ID for T5 special tokens
        # Apply masks using T5 special tokens
        for idx in mask_indices:
            tokens[idx] = f'<extra_id_{mask_id}>'
            mask_id += 1

        # Reconstruct the sentence with original spacing preserved
        masked_sentence = ''
        last_end = 0
        for i, offset in enumerate(token_offsets):
            start, end = offset
            masked_sentence += sentence[last_end:start] + tokens[i]
            last_end = end
        masked_sentence += sentence[last_end:]  # Add any trailing part of the sentence

        structure = '    '.join(structure_texts)

        return masked_sentence, structure

    def regenerate_text(self, masked_sentence):
        # Prepare the input data for the model
        input_text = "In clinical background, fill in the blank: " + masked_sentence
        inputs = self.tokenizer(input_text, return_tensors="pt").to(self.device)

        with torch.no_grad():
            outputs = self.model.generate(
                inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                max_length=512,
                num_beams=5,
                no_repeat_ngram_size=2,
                early_stopping=True
            )
            regenerated_sentence = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        masked_tokens = masked_sentence.split()
        predicted_tokens = regenerated_sentence.split()

        final_tokens = []
        pred_idx = 0  # Index for predicted tokens

        mask_token_pattern = re.compile(r'<extra_id_\d+>')

        for token in masked_tokens:
            if mask_token_pattern.match(token):
                self.total_masks += 1  # Count the total number of masks
                if pred_idx < len(predicted_tokens):
                    predicted_token = predicted_tokens[pred_idx]
                    final_tokens.append(predicted_token)
                    # Check if the predicted token is valid (alphanumeric and not in the dictionary)
                    if not re.match(r'^[a-zA-Z0-9-]+$', predicted_token):
                        self.invalid_masks += 1
                    pred_idx += 1
                else:
                    # Append the original token if no predicted token is available
                    final_tokens.append(token)
                    self.invalid_masks += 1
            else:
                # Append the original token if it is not a mask token
                final_tokens.append(token)

        final_output = ' '.join(final_tokens)
        return final_output


    def display_sentences_with_entities(self):
        unique_ids = self.merged_df['note_id'].drop_duplicates().sample(n=self.small_size, random_state=self.random_seed)
        for note_id in unique_ids:
            group = self.merged_df[self.merged_df['note_id'] == note_id]
            text = group['text'].iloc[0]
            annotations = group[['start', 'end', 'concept_id']].dropna().astype(int).to_numpy()
            self.process_text(note_id, text, annotations)

        # Calculate and display the invalid prediction mask ratio
        invalid_ratio = self.invalid_masks / self.total_masks if self.total_masks > 0 else 0
        # print("\n\033[1mInvalid Prediction Mask Ratio: \033[0m\n", invalid_ratio)
        print(f"\n\033[1mInvalid Prediction Mask Ratio: {invalid_ratio:.2f}\033[0m\n")


augmentor = MLMAugmentor(
    notes_path='/content/drive/MyDrive/MSc_Project/dataset/MimicIV/snomed_ct_entity_linking_challenge_1.0.0/snomed_ct_entity_linking_challenge_1.0.0/mimicIv_notes_training_set.csv',
    annotations_path='/content/drive/MyDrive/MSc_Project/dataset/MimicIV/snomed_ct_entity_linking_challenge_1.0.0/snomed_ct_entity_linking_challenge_1.0.0/train_annotations.csv',
    small_size=1,
    random_seed=42,
    masking_ratio=0.5,
    k=10
)
augmentor.display_sentences_with_entities()



Note ID: 10807423-DS-19, Sentence 1

Original Sentence:
  
Name:  ___               Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   M
 
Service: ORTHOPAEDICS
 
Allergies: 
No Known Allergies / Adverse Drug Reactions
 
Attending: ___.

Entities:
 [('No Known Allergies', 181, 199, 609328004), ('Adverse Drug Reactions', 202, 224, 419511003)]

Structure:
 Name :    Unit No :    Admission Date :    Discharge Date :    Date of Birth :    Sex :    M Service :    ORTHOPAEDICS Allergies :    No Known Allergies / Adverse Drug Reactions Attending :

Masked Sentence:
  
Name:  ___               Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   M
 
Service: ORTHOPAEDICS
 
Allergies: 
No Known Allergies / Adverse Drug Reactions
 
Attending: ___.

Generated Text:
 Name: ___ Unit No: ___ Admission Date: ___ Discharge Date: ___ Date of Birth: ___ Sex: M Service: ORTHOPAEDIC

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Metrics for Note ID: 10807423-DS-19, Sentence 1:
Metric                    Evaluation      Baseline       
--------------------------------------------------------------------------------
ROUGE-1                   100.00          100.00         
ROUGE-2                   100.00          100.00         
ROUGE-L                   100.00          100.00         
BERTScore F1              0.12            1.00           
METEOR                    1.00            1.00           
Flesch Reading Ease       51.52           Ori: 51.52, Mask: 51.52

Note ID: 10807423-DS-19, Sentence 2

Original Sentence:
 Chief Complaint:
R ankle fracture dislocation, open
 
Major Surgical or Invasive Procedure:
ORIF R ankle and I&D ___

 
History of Present Illness:
Chief Complaint:  ankle pain
Reason for Orthopedics Consult: management of open fracture
 
HISTORY OF PRESENT ILLNESS: 
Patient is a ___ yo male previously healhty presenting w/ fall 
from 6 feet, from ladder.

Entities:
 [('R ankle', 262, 269, 6685

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Metrics for Note ID: 10807423-DS-19, Sentence 2:
Metric                    Evaluation      Baseline       
--------------------------------------------------------------------------------
ROUGE-1                   86.79           73.77          
ROUGE-2                   75.00           63.33          
ROUGE-L                   84.91           73.77          
BERTScore F1              0.44            0.50           
METEOR                    0.85            0.85           
Flesch Reading Ease       8.21            Ori: 8.21, Mask: 8.21

Note ID: 10807423-DS-19, Sentence 3

Original Sentence:
 Patient landed on LLE w/ forced 
eversion and subsequent open fracture/dislocation. Denies head 
strike or LOC.

Entities:
 [('LLE', 621, 624, 32153003), ('eversion', 636, 644, 4196002), ('open fracture', 660, 673, 397181002), ('dislocation', 674, 685, 87642003), ('head \nstrike', 694, 706, 82271004), ('LOC', 710, 713, 419045004)]

Structure:
 

Masked Sentence:
 <mask> <mask> on LLE w/ <mask> 
e

...


Metrics for Note ID: 10807423-DS-19, Sentence 8:
Metric                    Evaluation      Baseline       
--------------------------------------------------------------------------------
ROUGE-1                   100.00          77.78          
ROUGE-2                   73.33           64.71          
ROUGE-L                   87.50           77.78          
BERTScore F1              0.51            0.60           
METEOR                    1.00            0.90           
Flesch Reading Ease       29.52           Ori: 29.52, Mask: 21.06

Note ID: 10807423-DS-19, Sentence 9

Original Sentence:
 PAST MEDICAL HISTORY:  
none
 
MEDICATIONS:  
none
 
ALLERGIES:  
NKDA
 
SOCIAL HISTORY:  
Denies alcohol, drugs, smoking
 
PHYSICAL EXAM:  
GENERAL: Alert, oriented, no acute distress  
HEENT: Sclera anicteric, MMM, oropharynx clear  
NECK: C-spine is non-tender to palpation
LUNGS: Clear to auscultation bilaterally
CV: Regular rate and rhythm, 
ABD: soft, non-tender, non-distended, 
PELVIS: st

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Metrics for Note ID: 10807423-DS-19, Sentence 9:
Metric                    Evaluation      Baseline       
--------------------------------------------------------------------------------
ROUGE-1                   91.73           80.82          
ROUGE-2                   83.97           73.61          
ROUGE-L                   91.73           80.82          
BERTScore F1              0.35            0.66           
METEOR                    0.92            0.90           
Flesch Reading Ease       -15.83          Ori: -15.83, Mask: -15.83

Note ID: 10807423-DS-19, Sentence 10

Original Sentence:
 +DP. Unable to assess. Warm, well perfused, 2+ pulses, 
no clubbing, cyanosis or edema.

Entities:
 []

Structure:
 

Masked Sentence:
 +DP. <mask> <mask> assess. Warm, <mask> <mask>, 2+ pulses, 
no clubbing, <mask> <mask> edema.

Generated Text:
 +DP. To No assess. Warm, edema no 2+ pulses, no clubbing, no -DP. edema.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Metrics for Note ID: 10807423-DS-19, Sentence 10:
Metric                    Evaluation      Baseline       
--------------------------------------------------------------------------------
ROUGE-1                   64.29           40.00          
ROUGE-2                   30.77           21.05          
ROUGE-L                   64.29           40.00          
BERTScore F1              0.32            0.02           
METEOR                    0.57            0.67           
Flesch Reading Ease       81.29           Ori: 81.29, Mask: 55.91

Invalid Prediction Mask Ratio: 0.20



Clinial T5

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
class MLMAugmentor:
    def __init__(self, notes_path, annotations_path, small_size=10, random_seed=42, masking_ratio=0.3, k=3, min_lines=3):
        self.notes_path = notes_path
        self.annotations_path = annotations_path
        self.small_size = small_size
        self.random_seed = random_seed
        self.masking_ratio = masking_ratio
        self.k = k
        self.min_lines = min_lines
        self.invalid_masks = 0
        self.total_masks = 0
        random.seed(self.random_seed)  # for reproducibility
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/MSc_Project/Models/Clinical_T5_Base')
        self.model = AutoModelForSeq2SeqLM.from_pretrained('/content/drive/MyDrive/MSc_Project/Models/Clinical_T5_Base').to(self.device)
        # self.scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
        self.load_and_merge_datasets()
        self.rouge = evaluate_load('rouge')

    def load_and_merge_datasets(self):
        try:
            notes_df = pd.read_csv(self.notes_path)
            annotations_df = pd.read_csv(self.annotations_path)
            self.merged_df = pd.merge(notes_df, annotations_df, on='note_id', how='left')
        except Exception as e:
            print(f"An error occurred: {e}")

    def split_into_sentences(self, text, min_lines=3):
        # Splitting text into sentence
        sentences = sent_tokenize(text)
        processed_sentences = []
        current_block = []
        num_lines = 0


        for sentence in sentences:
            current_block.append(sentence)
            num_lines += sentence.count('\n') + 1

            if num_lines >= min_lines:
                processed_sentences.append(" ".join(current_block))
                current_block = []
                num_lines = 0

        # Add the last block if it's not empty
        if current_block:
            processed_sentences.append(" ".join(current_block))

        return processed_sentences

    def process_text(self, note_id, text, annotations):
        sentences = self.split_into_sentences(text, self.min_lines)
        start_index = 0
        for i, sentence in enumerate(sentences[:self.k]):
            sentence_start = text.find(sentence)
            sentence_end = sentence_start + len(sentence)
            entities = [(text[s:e], s, e, cid) for s, e, cid in annotations if s >= sentence_start and e <= sentence_end]
            masked_sentence, structure = self.mask_text(sentence, entities, sentence_start, sentence_end)
            regenerated_sentence = self.regenerate_text(masked_sentence)

            # Replace <extra_id_x> with <mask> for printing
            display_masked_sentence = re.sub(r'<extra_id_\d+>', '<mask>', masked_sentence)
            print(f"\n\033[1mNote ID: {note_id}, Sentence {i+1}\033[0m")
            print("\n\033[1mOriginal Sentence:\033[0m\n", sentence)
            print("\n\033[1mEntities:\033[0m\n", entities)
            print("\n\033[1mStructure:\033[0m\n", structure)
            print("\n\033[1mMasked Sentence:\033[0m\n", display_masked_sentence)
            print("\n\033[1mGenerated Text:\033[0m\n", regenerated_sentence)

            # # Compute ROUGE-L score
            # rouge_scores = self.scorer.score(sentence, regenerated_sentence)['rougeL']
            # print("\n\033[1mROUGE-L Score:\033[0m")
            # print(f"Precision: {rouge_scores.precision:.4f}, Recall: {rouge_scores.recall:.4f}, F1: {rouge_scores.fmeasure:.4f}")

            # # Compute BERTScore using 'bert-base-uncased' model
            # P, R, F1 = bert_score_fn([regenerated_sentence], [sentence], lang="en", model_type="bert-base-uncased", rescale_with_baseline=True)
            # # P, R, F1 = bert_score_fn([regenerated_sentence], [sentence], lang="en", model_type="bert-base-uncased", rescale_with_baseline=True, show_progress_bar=False)
            # print("\n\033[1mBERTScore:\033[0m")
            # print(f"Precision: {P.mean().item():.4f}, Recall: {R.mean().item():.4f}, F1: {F1.mean().item():.4f}")
            rouge_results = self.rouge.compute(predictions=[regenerated_sentence], references=[sentence])
            P, R, F1 = bert_score_fn([regenerated_sentence], [sentence], lang="en", rescale_with_baseline=True)

            rouge_1 = rouge_results['rouge1'] * 100
            rouge_2 = rouge_results['rouge2'] * 100
            rouge_L = rouge_results['rougeL'] * 100
            bertscore_f1 = F1.mean().cpu().item()
            flesch_reading_ease = textstat.flesch_reading_ease(regenerated_sentence)

            # METEOR score
            tokenized_sentence = word_tokenize(sentence)
            tokenized_regeneration = word_tokenize(regenerated_sentence)
            meteor = meteor_score([tokenized_sentence], tokenized_regeneration)

            flesch_reading_ease_ori = textstat.flesch_reading_ease(sentence)
            flesch_reading_ease_mask = textstat.flesch_reading_ease(masked_sentence)
            rouge_results_mask = self.rouge.compute(predictions=[masked_sentence], references=[sentence])
            rouge_1_mask = rouge_results_mask['rouge1'] * 100
            rouge_2_mask = rouge_results_mask['rouge2'] * 100
            rouge_L_mask = rouge_results_mask['rougeL'] * 100

            P_Mask, R_Mask, F1_Mask = bert_score_fn([masked_sentence], [sentence], lang="en", rescale_with_baseline=True)
            bertscore_f1_mask = F1_Mask.mean().cpu().item()

            # METEOR score
            tokenized_masked_sentence = word_tokenize(masked_sentence)
            meteor_mask = meteor_score([tokenized_sentence], tokenized_masked_sentence)

            print(f"\n\033[1mMetrics for Note ID: {note_id}, Sentence {i+1}\033[0m:")
            print("{:<25} {:<15} {:<15}".format("Metric", "Evaluation", "Baseline"))
            print("-" * 80)
            print("{:<25} {:<15.2f} {:<15.2f}".format("ROUGE-1", rouge_1, rouge_1_mask))
            print("{:<25} {:<15.2f} {:<15.2f}".format("ROUGE-2", rouge_2, rouge_2_mask))
            print("{:<25} {:<15.2f} {:<15.2f}".format("ROUGE-L", rouge_L, rouge_L_mask))
            print("{:<25} {:<15.2f} {:<15.2f}".format("BERTScore F1", bertscore_f1, bertscore_f1_mask))
            print("{:<25} {:<15.2f} {:<15.2f}".format("METEOR", meteor, meteor_mask))
            print("{:<25} {:<15.2f} {:<15}".format("Flesch Reading Ease", flesch_reading_ease, f"Ori: {flesch_reading_ease_ori:.2f}, Mask: {flesch_reading_ease_mask:.2f}"))


            print("="*80)

    def mask_text(self, sentence, entities, sentence_start, sentence_end):
        tokens = word_tokenize(sentence)
        original_tokens = tokens[:]  # Copy of original tokens
        mask = [True] * len(tokens)  # Initialize mask list, assuming all tokens can be masked

        # Generate character offsets for each token using word_tokenize positions
        token_offsets = []
        current_position = 0
        for token in tokens:
            start_position = sentence.find(token, current_position)
            end_position = start_position + len(token)
            token_offsets.append((start_position, end_position))
            current_position = end_position

        structure_texts = []
        allowed_lower = {'a', 'the', 'of', 'in', 'on', 'at', 'or', 'by', 'for', 'with', 'about', 'as', 'an', 'to'}
        termination_punctuation = {'.', '_', '___', '!', '?', ':'}
        # non_termination_pattern = re.compile(r'[^a-zA-Z\.!_?___]')
        non_termination_pattern = re.compile(r'[^.!:_?___]')

        i = 0
        while i < len(tokens):
            if tokens[i].endswith(':'):
                # Start checking backwards from the colon
                title_start = i - 1
                while title_start >= 0:
                    current_token = tokens[title_start]
                    if current_token in termination_punctuation or (current_token.islower() and current_token not in allowed_lower):
                        break  # Stop if a termination punctuation or a non-allowed lowercase word is encountered
                    title_start -= 1
                title_start += 1  # Adjust to include the first valid word after the break point

                # Capture the title segment if valid
                if title_start < i:
                    title_segment = tokens[title_start:i + 1]  # Include the colon in the title
                    structure_text = ' '.join(title_segment)
                    if not structure_texts or (structure_texts and structure_text not in structure_texts[-1]):
                        structure_texts.append(structure_text)
                        # Mask the title segment
                        for j in range(title_start, i + 1):
                            mask[j] = False
                i = i + 1  # Ensure to move past the last processed colon
            else:
                i += 1

        # Protect entities based on their positions and non-alphabetic tokens
        special_pattern = re.compile(r'\w\s*[^\w\s]\s*\d')
        for idx, token in enumerate(tokens):
            if not token.isalpha() or token.isupper() or re.search(r'\d', token) or special_pattern.search(token):  # Check if the token is purely alphabetic
                mask[idx] = False

        # only mask entities (identify borders accuratly)
        for _, ent_start, ent_end, _ in entities:
            for idx, (token_start, token_end) in enumerate(token_offsets):
                if (token_start + sentence_start >= ent_start and token_start + sentence_start < ent_end) or \
                  (token_end + sentence_start > ent_start and token_end + sentence_start <= ent_end) or \
                  (token_start + sentence_start <= ent_start and token_end + sentence_start >= ent_end):
                    mask[idx] = False

        for match in re.finditer(r'\w\s*[^\w\s]\s*\d', sentence):
            match_start, match_end = match.span()
            for idx, (token_start, token_end) in enumerate(token_offsets):
                if token_start >= match_start and token_end <= match_end:
                    mask[idx] = False
                    break

        # Calculate the indices of maskable tokens
        maskable_indices = [idx for idx, m in enumerate(mask) if m]
        if self.masking_ratio == 1.0:
            # If mask_ratio is 1.0, mask all maskable tokens
            mask_indices = maskable_indices
        else:
            num_to_mask = int(len(maskable_indices) * self.masking_ratio)
            mask_indices = random.sample(maskable_indices, num_to_mask) if maskable_indices else []

        mask_id = 0  # Initialize mask ID for T5 special tokens
        # Apply masks using T5 special tokens
        for idx in mask_indices:
            tokens[idx] = f'<extra_id_{mask_id}>'
            mask_id += 1

        # Reconstruct the sentence with original spacing preserved
        masked_sentence = ''
        last_end = 0
        for i, offset in enumerate(token_offsets):
            start, end = offset
            masked_sentence += sentence[last_end:start] + tokens[i]
            last_end = end
        masked_sentence += sentence[last_end:]  # Add any trailing part of the sentence

        structure = '    '.join(structure_texts)

        return masked_sentence, structure

    def regenerate_text(self, masked_sentence):
        # Prepare the input data for the model
        input_text = "In clinical background, fill in the blank: " + masked_sentence
        inputs = self.tokenizer(input_text, return_tensors="pt").to(self.device)

        with torch.no_grad():
            outputs = self.model.generate(
                inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                max_length=512,
                num_beams=5,
                no_repeat_ngram_size=2,
                early_stopping=True
            )
            regenerated_sentence = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        masked_tokens = masked_sentence.split()
        predicted_tokens = regenerated_sentence.split()

        final_tokens = []
        pred_idx = 0  # Index for predicted tokens

        mask_token_pattern = re.compile(r'<extra_id_\d+>')

        for token in masked_tokens:
            if mask_token_pattern.match(token):
                self.total_masks += 1  # Count the total number of masks
                if pred_idx < len(predicted_tokens):
                    predicted_token = predicted_tokens[pred_idx]
                    final_tokens.append(predicted_token)
                    # Check if the predicted token is valid (alphanumeric and not in the dictionary)
                    if not re.match(r'^[a-zA-Z0-9-]+$', predicted_token):
                        self.invalid_masks += 1
                    pred_idx += 1
                else:
                    # Append the original token if no predicted token is available
                    final_tokens.append(token)
                    self.invalid_masks += 1
            else:
                # Append the original token if it is not a mask token
                final_tokens.append(token)

        final_output = ' '.join(final_tokens)
        return final_output


    def display_sentences_with_entities(self):
        unique_ids = self.merged_df['note_id'].drop_duplicates().sample(n=self.small_size, random_state=self.random_seed)
        for note_id in unique_ids:
            group = self.merged_df[self.merged_df['note_id'] == note_id]
            text = group['text'].iloc[0]
            annotations = group[['start', 'end', 'concept_id']].dropna().astype(int).to_numpy()
            self.process_text(note_id, text, annotations)

        # Calculate and display the invalid prediction mask ratio
        invalid_ratio = self.invalid_masks / self.total_masks if self.total_masks > 0 else 0
        # print("\n\033[1mInvalid Prediction Mask Ratio: \033[0m\n", invalid_ratio)
        print(f"\n\033[1mInvalid Prediction Mask Ratio: {invalid_ratio:.2f}\033[0m\n")


augmentor = MLMAugmentor(
    notes_path='/content/drive/MyDrive/MSc_Project/dataset/MimicIV/snomed_ct_entity_linking_challenge_1.0.0/snomed_ct_entity_linking_challenge_1.0.0/mimicIv_notes_training_set.csv',
    annotations_path='/content/drive/MyDrive/MSc_Project/dataset/MimicIV/snomed_ct_entity_linking_challenge_1.0.0/snomed_ct_entity_linking_challenge_1.0.0/train_annotations.csv',
    small_size=1,
    random_seed=42,
    masking_ratio=0.5,
    k=10
)
augmentor.display_sentences_with_entities()



Note ID: 10807423-DS-19, Sentence 1

Original Sentence:
  
Name:  ___               Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   M
 
Service: ORTHOPAEDICS
 
Allergies: 
No Known Allergies / Adverse Drug Reactions
 
Attending: ___.

Entities:
 [('No Known Allergies', 181, 199, 609328004), ('Adverse Drug Reactions', 202, 224, 419511003)]

Structure:
 Name :    Unit No :    Admission Date :    Discharge Date :    Date of Birth :    Sex :    M Service :    ORTHOPAEDICS Allergies :    No Known Allergies / Adverse Drug Reactions Attending :

Masked Sentence:
  
Name:  ___               Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   M
 
Service: ORTHOPAEDICS
 
Allergies: 
No Known Allergies / Adverse Drug Reactions
 
Attending: ___.

Generated Text:
 Name: ___ Unit No: ___ Admission Date: ___ Discharge Date: ___ Date of Birth: ___ Sex: M Service: ORTHOPAEDIC

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Metrics for Note ID: 10807423-DS-19, Sentence 1:
Metric                    Evaluation      Baseline       
--------------------------------------------------------------------------------
ROUGE-1                   100.00          100.00         
ROUGE-2                   100.00          100.00         
ROUGE-L                   100.00          100.00         
BERTScore F1              0.12            1.00           
METEOR                    1.00            1.00           
Flesch Reading Ease       51.52           Ori: 51.52, Mask: 51.52

Note ID: 10807423-DS-19, Sentence 2

Original Sentence:
 Chief Complaint:
R ankle fracture dislocation, open
 
Major Surgical or Invasive Procedure:
ORIF R ankle and I&D ___

 
History of Present Illness:
Chief Complaint:  ankle pain
Reason for Orthopedics Consult: management of open fracture
 
HISTORY OF PRESENT ILLNESS: 
Patient is a ___ yo male previously healhty presenting w/ fall 
from 6 feet, from ladder.

Entities:
 [('R ankle', 262, 269, 6685

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Metrics for Note ID: 10807423-DS-19, Sentence 2:
Metric                    Evaluation      Baseline       
--------------------------------------------------------------------------------
ROUGE-1                   85.19           73.77          
ROUGE-2                   71.70           63.33          
ROUGE-L                   83.33           73.77          
BERTScore F1              0.40            0.50           
METEOR                    0.83            0.85           
Flesch Reading Ease       8.21            Ori: 8.21, Mask: 8.21

Note ID: 10807423-DS-19, Sentence 3

Original Sentence:
 Patient landed on LLE w/ forced 
eversion and subsequent open fracture/dislocation. Denies head 
strike or LOC.

Entities:
 [('LLE', 621, 624, 32153003), ('eversion', 636, 644, 4196002), ('open fracture', 660, 673, 397181002), ('dislocation', 674, 685, 87642003), ('head \nstrike', 694, 706, 82271004), ('LOC', 710, 713, 419045004)]

Structure:
 

Masked Sentence:
 <mask> <mask> on LLE w/ <mask> 
e

...


Metrics for Note ID: 10807423-DS-19, Sentence 8:
Metric                    Evaluation      Baseline       
--------------------------------------------------------------------------------
ROUGE-1                   93.75           77.78          
ROUGE-2                   86.67           64.71          
ROUGE-L                   93.75           77.78          
BERTScore F1              0.48            0.60           
METEOR                    0.94            0.90           
Flesch Reading Ease       29.52           Ori: 29.52, Mask: 21.06

Note ID: 10807423-DS-19, Sentence 9

Original Sentence:
 PAST MEDICAL HISTORY:  
none
 
MEDICATIONS:  
none
 
ALLERGIES:  
NKDA
 
SOCIAL HISTORY:  
Denies alcohol, drugs, smoking
 
PHYSICAL EXAM:  
GENERAL: Alert, oriented, no acute distress  
HEENT: Sclera anicteric, MMM, oropharynx clear  
NECK: C-spine is non-tender to palpation
LUNGS: Clear to auscultation bilaterally
CV: Regular rate and rhythm, 
ABD: soft, non-tender, non-distended, 
PELVIS: st

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Metrics for Note ID: 10807423-DS-19, Sentence 9:
Metric                    Evaluation      Baseline       
--------------------------------------------------------------------------------
ROUGE-1                   88.41           80.82          
ROUGE-2                   77.94           73.61          
ROUGE-L                   85.51           80.82          
BERTScore F1              0.32            0.66           
METEOR                    0.87            0.90           
Flesch Reading Ease       -24.29          Ori: -15.83, Mask: -15.83

Note ID: 10807423-DS-19, Sentence 10

Original Sentence:
 +DP. Unable to assess. Warm, well perfused, 2+ pulses, 
no clubbing, cyanosis or edema.

Entities:
 []

Structure:
 

Masked Sentence:
 +DP. <mask> <mask> assess. Warm, <mask> <mask>, 2+ pulses, 
no clubbing, <mask> <mask> edema.

Generated Text:
 +DP. [MISC] +[Reg#][#] assess. Warm, Difficult to[Name] 2+ pulses, no clubbing, No[DAY] well edema.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Metrics for Note ID: 10807423-DS-19, Sentence 10:
Metric                    Evaluation      Baseline       
--------------------------------------------------------------------------------
ROUGE-1                   66.67           40.00          
ROUGE-2                   28.57           21.05          
ROUGE-L                   53.33           40.00          
BERTScore F1              0.20            0.02           
METEOR                    0.65            0.67           
Flesch Reading Ease       72.83           Ori: 81.29, Mask: 55.91

Invalid Prediction Mask Ratio: 0.70



Clinical T5 Scratch

In [ ]:
class MLMAugmentor:
    def __init__(self, notes_path, annotations_path, small_size=10, random_seed=42, masking_ratio=0.3, k=3, min_lines=3):
        self.notes_path = notes_path
        self.annotations_path = annotations_path
        self.small_size = small_size
        self.random_seed = random_seed
        self.masking_ratio = masking_ratio
        self.k = k
        self.min_lines = min_lines
        self.invalid_masks = 0
        self.total_masks = 0
        random.seed(self.random_seed)  # for reproducibility
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/MSc_Project/Models/Clinical_T5_Scratch')
        self.model = AutoModelForSeq2SeqLM.from_pretrained('/content/drive/MyDrive/MSc_Project/Models/Clinical_T5_Scratch').to(self.device)
        # self.scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
        self.load_and_merge_datasets()
        self.rouge = evaluate_load('rouge')

    def load_and_merge_datasets(self):
        try:
            notes_df = pd.read_csv(self.notes_path)
            annotations_df = pd.read_csv(self.annotations_path)
            self.merged_df = pd.merge(notes_df, annotations_df, on='note_id', how='left')
        except Exception as e:
            print(f"An error occurred: {e}")

    def split_into_sentences(self, text, min_lines=3):
        # Splitting text into sentence
        sentences = sent_tokenize(text)
        processed_sentences = []
        current_block = []
        num_lines = 0


        for sentence in sentences:
            current_block.append(sentence)
            num_lines += sentence.count('\n') + 1

            if num_lines >= min_lines:
                processed_sentences.append(" ".join(current_block))
                current_block = []
                num_lines = 0

        # Add the last block if it's not empty
        if current_block:
            processed_sentences.append(" ".join(current_block))

        return processed_sentences

    def process_text(self, note_id, text, annotations):
        sentences = self.split_into_sentences(text, self.min_lines)
        start_index = 0
        for i, sentence in enumerate(sentences[:self.k]):
            sentence_start = text.find(sentence)
            sentence_end = sentence_start + len(sentence)
            entities = [(text[s:e], s, e, cid) for s, e, cid in annotations if s >= sentence_start and e <= sentence_end]
            masked_sentence, structure = self.mask_text(sentence, entities, sentence_start, sentence_end)
            regenerated_sentence = self.regenerate_text(masked_sentence)

            # Replace <extra_id_x> with <mask> for printing
            display_masked_sentence = re.sub(r'<extra_id_\d+>', '<mask>', masked_sentence)
            print(f"\n\033[1mNote ID: {note_id}, Sentence {i+1}\033[0m")
            print("\n\033[1mOriginal Sentence:\033[0m\n", sentence)
            print("\n\033[1mEntities:\033[0m\n", entities)
            print("\n\033[1mStructure:\033[0m\n", structure)
            print("\n\033[1mMasked Sentence:\033[0m\n", display_masked_sentence)
            print("\n\033[1mGenerated Text:\033[0m\n", regenerated_sentence)

            # # Compute ROUGE-L score
            # rouge_scores = self.scorer.score(sentence, regenerated_sentence)['rougeL']
            # print("\n\033[1mROUGE-L Score:\033[0m")
            # print(f"Precision: {rouge_scores.precision:.4f}, Recall: {rouge_scores.recall:.4f}, F1: {rouge_scores.fmeasure:.4f}")

            # # Compute BERTScore using 'bert-base-uncased' model
            # P, R, F1 = bert_score_fn([regenerated_sentence], [sentence], lang="en", model_type="bert-base-uncased", rescale_with_baseline=True)
            # # P, R, F1 = bert_score_fn([regenerated_sentence], [sentence], lang="en", model_type="bert-base-uncased", rescale_with_baseline=True, show_progress_bar=False)
            # print("\n\033[1mBERTScore:\033[0m")
            # print(f"Precision: {P.mean().item():.4f}, Recall: {R.mean().item():.4f}, F1: {F1.mean().item():.4f}")
            rouge_results = self.rouge.compute(predictions=[regenerated_sentence], references=[sentence])
            P, R, F1 = bert_score_fn([regenerated_sentence], [sentence], lang="en", rescale_with_baseline=True)

            rouge_1 = rouge_results['rouge1'] * 100
            rouge_2 = rouge_results['rouge2'] * 100
            rouge_L = rouge_results['rougeL'] * 100
            bertscore_f1 = F1.mean().cpu().item()
            flesch_reading_ease = textstat.flesch_reading_ease(regenerated_sentence)

            # METEOR score
            tokenized_sentence = word_tokenize(sentence)
            tokenized_regeneration = word_tokenize(regenerated_sentence)
            meteor = meteor_score([tokenized_sentence], tokenized_regeneration)

            flesch_reading_ease_ori = textstat.flesch_reading_ease(sentence)
            flesch_reading_ease_mask = textstat.flesch_reading_ease(masked_sentence)
            rouge_results_mask = self.rouge.compute(predictions=[masked_sentence], references=[sentence])
            rouge_1_mask = rouge_results_mask['rouge1'] * 100
            rouge_2_mask = rouge_results_mask['rouge2'] * 100
            rouge_L_mask = rouge_results_mask['rougeL'] * 100

            P_Mask, R_Mask, F1_Mask = bert_score_fn([masked_sentence], [sentence], lang="en", rescale_with_baseline=True)
            bertscore_f1_mask = F1_Mask.mean().cpu().item()

            # METEOR score
            tokenized_masked_sentence = word_tokenize(masked_sentence)
            meteor_mask = meteor_score([tokenized_sentence], tokenized_masked_sentence)

            print(f"\n\033[1mMetrics for Note ID: {note_id}, Sentence {i+1}\033[0m:")
            print("{:<25} {:<15} {:<15}".format("Metric", "Evaluation", "Baseline"))
            print("-" * 80)
            print("{:<25} {:<15.2f} {:<15.2f}".format("ROUGE-1", rouge_1, rouge_1_mask))
            print("{:<25} {:<15.2f} {:<15.2f}".format("ROUGE-2", rouge_2, rouge_2_mask))
            print("{:<25} {:<15.2f} {:<15.2f}".format("ROUGE-L", rouge_L, rouge_L_mask))
            print("{:<25} {:<15.2f} {:<15.2f}".format("BERTScore F1", bertscore_f1, bertscore_f1_mask))
            print("{:<25} {:<15.2f} {:<15.2f}".format("METEOR", meteor, meteor_mask))
            print("{:<25} {:<15.2f} {:<15}".format("Flesch Reading Ease", flesch_reading_ease, f"Ori: {flesch_reading_ease_ori:.2f}, Mask: {flesch_reading_ease_mask:.2f}"))


            print("="*80)

    def mask_text(self, sentence, entities, sentence_start, sentence_end):
        tokens = word_tokenize(sentence)
        original_tokens = tokens[:]  # Copy of original tokens
        mask = [True] * len(tokens)  # Initialize mask list, assuming all tokens can be masked

        # Generate character offsets for each token using word_tokenize positions
        token_offsets = []
        current_position = 0
        for token in tokens:
            start_position = sentence.find(token, current_position)
            end_position = start_position + len(token)
            token_offsets.append((start_position, end_position))
            current_position = end_position

        structure_texts = []
        allowed_lower = {'a', 'the', 'of', 'in', 'on', 'at', 'or', 'by', 'for', 'with', 'about', 'as', 'an', 'to'}
        termination_punctuation = {'.', '_', '___', '!', '?', ':'}
        # non_termination_pattern = re.compile(r'[^a-zA-Z\.!_?___]')
        non_termination_pattern = re.compile(r'[^.!:_?___]')

        i = 0
        while i < len(tokens):
            if tokens[i].endswith(':'):
                # Start checking backwards from the colon
                title_start = i - 1
                while title_start >= 0:
                    current_token = tokens[title_start]
                    if current_token in termination_punctuation or (current_token.islower() and current_token not in allowed_lower):
                        break  # Stop if a termination punctuation or a non-allowed lowercase word is encountered
                    title_start -= 1
                title_start += 1  # Adjust to include the first valid word after the break point

                # Capture the title segment if valid
                if title_start < i:
                    title_segment = tokens[title_start:i + 1]  # Include the colon in the title
                    structure_text = ' '.join(title_segment)
                    if not structure_texts or (structure_texts and structure_text not in structure_texts[-1]):
                        structure_texts.append(structure_text)
                        # Mask the title segment
                        for j in range(title_start, i + 1):
                            mask[j] = False
                i = i + 1  # Ensure to move past the last processed colon
            else:
                i += 1

        # Protect entities based on their positions and non-alphabetic tokens
        special_pattern = re.compile(r'\w\s*[^\w\s]\s*\d')
        for idx, token in enumerate(tokens):
            if not token.isalpha() or token.isupper() or re.search(r'\d', token) or special_pattern.search(token):  # Check if the token is purely alphabetic
                mask[idx] = False

        # only mask entities (identify borders accuratly)
        for _, ent_start, ent_end, _ in entities:
            for idx, (token_start, token_end) in enumerate(token_offsets):
                if (token_start + sentence_start >= ent_start and token_start + sentence_start < ent_end) or \
                  (token_end + sentence_start > ent_start and token_end + sentence_start <= ent_end) or \
                  (token_start + sentence_start <= ent_start and token_end + sentence_start >= ent_end):
                    mask[idx] = False

        for match in re.finditer(r'\w\s*[^\w\s]\s*\d', sentence):
            match_start, match_end = match.span()
            for idx, (token_start, token_end) in enumerate(token_offsets):
                if token_start >= match_start and token_end <= match_end:
                    mask[idx] = False
                    break

        # Calculate the indices of maskable tokens
        maskable_indices = [idx for idx, m in enumerate(mask) if m]
        if self.masking_ratio == 1.0:
            # If mask_ratio is 1.0, mask all maskable tokens
            mask_indices = maskable_indices
        else:
            num_to_mask = int(len(maskable_indices) * self.masking_ratio)
            mask_indices = random.sample(maskable_indices, num_to_mask) if maskable_indices else []

        mask_id = 0  # Initialize mask ID for T5 special tokens
        # Apply masks using T5 special tokens
        for idx in mask_indices:
            tokens[idx] = f'<extra_id_{mask_id}>'
            mask_id += 1

        # Reconstruct the sentence with original spacing preserved
        masked_sentence = ''
        last_end = 0
        for i, offset in enumerate(token_offsets):
            start, end = offset
            masked_sentence += sentence[last_end:start] + tokens[i]
            last_end = end
        masked_sentence += sentence[last_end:]  # Add any trailing part of the sentence

        structure = '    '.join(structure_texts)

        return masked_sentence, structure

    def regenerate_text(self, masked_sentence):
        # Prepare the input data for the model
        input_text = "In clinical background, fill in the blank: " + masked_sentence
        inputs = self.tokenizer(input_text, return_tensors="pt").to(self.device)

        with torch.no_grad():
            outputs = self.model.generate(
                inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                max_length=512,
                num_beams=5,
                no_repeat_ngram_size=2,
                early_stopping=True
            )
            regenerated_sentence = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        masked_tokens = masked_sentence.split()
        predicted_tokens = regenerated_sentence.split()

        final_tokens = []
        pred_idx = 0  # Index for predicted tokens

        mask_token_pattern = re.compile(r'<extra_id_\d+>')

        for token in masked_tokens:
            if mask_token_pattern.match(token):
                self.total_masks += 1  # Count the total number of masks
                if pred_idx < len(predicted_tokens):
                    predicted_token = predicted_tokens[pred_idx]
                    final_tokens.append(predicted_token)
                    # Check if the predicted token is valid (alphanumeric and not in the dictionary)
                    if not re.match(r'^[a-zA-Z0-9-]+$', predicted_token):
                        self.invalid_masks += 1
                    pred_idx += 1
                else:
                    # Append the original token if no predicted token is available
                    final_tokens.append(token)
                    self.invalid_masks += 1
            else:
                # Append the original token if it is not a mask token
                final_tokens.append(token)

        final_output = ' '.join(final_tokens)
        return final_output


    def display_sentences_with_entities(self):
        unique_ids = self.merged_df['note_id'].drop_duplicates().sample(n=self.small_size, random_state=self.random_seed)
        for note_id in unique_ids:
            group = self.merged_df[self.merged_df['note_id'] == note_id]
            text = group['text'].iloc[0]
            annotations = group[['start', 'end', 'concept_id']].dropna().astype(int).to_numpy()
            self.process_text(note_id, text, annotations)

        # Calculate and display the invalid prediction mask ratio
        invalid_ratio = self.invalid_masks / self.total_masks if self.total_masks > 0 else 0
        # print("\n\033[1mInvalid Prediction Mask Ratio: \033[0m\n", invalid_ratio)
        print(f"\n\033[1mInvalid Prediction Mask Ratio: {invalid_ratio:.2f}\033[0m\n")


augmentor = MLMAugmentor(
    notes_path='/content/drive/MyDrive/MSc_Project/dataset/MimicIV/snomed_ct_entity_linking_challenge_1.0.0/snomed_ct_entity_linking_challenge_1.0.0/mimicIv_notes_training_set.csv',
    annotations_path='/content/drive/MyDrive/MSc_Project/dataset/MimicIV/snomed_ct_entity_linking_challenge_1.0.0/snomed_ct_entity_linking_challenge_1.0.0/train_annotations.csv',
    small_size=1,
    random_seed=42,
    masking_ratio=0.5,
    k=10
)
augmentor.display_sentences_with_entities()


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



Note ID: 10807423-DS-19, Sentence 1

Original Sentence:
  
Name:  ___               Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   M
 
Service: ORTHOPAEDICS
 
Allergies: 
No Known Allergies / Adverse Drug Reactions
 
Attending: ___.

Entities:
 [('No Known Allergies', 181, 199, 609328004), ('Adverse Drug Reactions', 202, 224, 419511003)]

Structure:
 Name :    Unit No :    Admission Date :    Discharge Date :    Date of Birth :    Sex :    M Service :    ORTHOPAEDICS Allergies :    No Known Allergies / Adverse Drug Reactions Attending :

Masked Sentence:
  
Name:  ___               Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   M
 
Service: ORTHOPAEDICS
 
Allergies: 
No Known Allergies / Adverse Drug Reactions
 
Attending: ___.

Generated Text:
 Name: ___ Unit No: ___ Admission Date: ___ Discharge Date: ___ Date of Birth: ___ Sex: M Service: ORTHOPAEDIC

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Metrics for Note ID: 10807423-DS-19, Sentence 1:
Metric                    Evaluation      Baseline       
--------------------------------------------------------------------------------
ROUGE-1                   100.00          100.00         
ROUGE-2                   100.00          100.00         
ROUGE-L                   100.00          100.00         
BERTScore F1              0.12            1.00           
METEOR                    1.00            1.00           
Flesch Reading Ease       51.52           Ori: 51.52, Mask: 51.52

Note ID: 10807423-DS-19, Sentence 2

Original Sentence:
 Chief Complaint:
R ankle fracture dislocation, open
 
Major Surgical or Invasive Procedure:
ORIF R ankle and I&D ___

 
History of Present Illness:
Chief Complaint:  ankle pain
Reason for Orthopedics Consult: management of open fracture
 
HISTORY OF PRESENT ILLNESS: 
Patient is a ___ yo male previously healhty presenting w/ fall 
from 6 feet, from ladder.

Entities:
 [('R ankle', 262, 269, 6685

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...


Metrics for Note ID: 10807423-DS-19, Sentence 8:
Metric                    Evaluation      Baseline       
--------------------------------------------------------------------------------
ROUGE-1                   93.33           77.78          
ROUGE-2                   78.57           64.71          
ROUGE-L                   93.33           77.78          
BERTScore F1              0.47            0.60           
METEOR                    0.91            0.90           
Flesch Reading Ease       22.07           Ori: 29.52, Mask: 21.06

Note ID: 10807423-DS-19, Sentence 9

Original Sentence:
 PAST MEDICAL HISTORY:  
none
 
MEDICATIONS:  
none
 
ALLERGIES:  
NKDA
 
SOCIAL HISTORY:  
Denies alcohol, drugs, smoking
 
PHYSICAL EXAM:  
GENERAL: Alert, oriented, no acute distress  
HEENT: Sclera anicteric, MMM, oropharynx clear  
NECK: C-spine is non-tender to palpation
LUNGS: Clear to auscultation bilaterally
CV: Regular rate and rhythm, 
ABD: soft, non-tender, non-distended, 
PELVIS: st

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Metrics for Note ID: 10807423-DS-19, Sentence 9:
Metric                    Evaluation      Baseline       
--------------------------------------------------------------------------------
ROUGE-1                   92.19           80.82          
ROUGE-2                   84.13           73.61          
ROUGE-L                   92.19           80.82          
BERTScore F1              0.33            0.66           
METEOR                    0.81            0.90           
Flesch Reading Ease       -11.77          Ori: -15.83, Mask: -15.83

Note ID: 10807423-DS-19, Sentence 10

Original Sentence:
 +DP. Unable to assess. Warm, well perfused, 2+ pulses, 
no clubbing, cyanosis or edema.

Entities:
 []

Structure:
 

Masked Sentence:
 +DP. <mask> <mask> assess. Warm, <mask> <mask>, 2+ pulses, 
no clubbing, <mask> <mask> edema.

Generated Text:
 +DP. : : assess. Warm, : : 2+ pulses, no clubbing, , [name, edema.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Metrics for Note ID: 10807423-DS-19, Sentence 10:
Metric                    Evaluation      Baseline       
--------------------------------------------------------------------------------
ROUGE-1                   69.57           40.00          
ROUGE-2                   38.10           21.05          
ROUGE-L                   69.57           40.00          
BERTScore F1              0.35            0.02           
METEOR                    0.63            0.67           
Flesch Reading Ease       79.26           Ori: 81.29, Mask: 55.91

Invalid Prediction Mask Ratio: 0.82



Clinical-T5-Sci

In [ ]:
class MLMAugmentor:
    def __init__(self, notes_path, annotations_path, small_size=10, random_seed=42, masking_ratio=0.3, k=3, min_lines=3):
        self.notes_path = notes_path
        self.annotations_path = annotations_path
        self.small_size = small_size
        self.random_seed = random_seed
        self.masking_ratio = masking_ratio
        self.k = k
        self.min_lines = min_lines
        self.invalid_masks = 0
        self.total_masks = 0
        random.seed(self.random_seed)  # for reproducibility
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/MSc_Project/Models/Clinical_T5_Sci')
        self.model = AutoModelForSeq2SeqLM.from_pretrained('/content/drive/MyDrive/MSc_Project/Models/Clinical_T5_Sci').to(self.device)
        # self.scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
        self.load_and_merge_datasets()
        self.rouge = evaluate_load('rouge')

    def load_and_merge_datasets(self):
        try:
            notes_df = pd.read_csv(self.notes_path)
            annotations_df = pd.read_csv(self.annotations_path)
            self.merged_df = pd.merge(notes_df, annotations_df, on='note_id', how='left')
        except Exception as e:
            print(f"An error occurred: {e}")

    def split_into_sentences(self, text, min_lines=3):
        # Splitting text into sentence
        sentences = sent_tokenize(text)
        processed_sentences = []
        current_block = []
        num_lines = 0


        for sentence in sentences:
            current_block.append(sentence)
            num_lines += sentence.count('\n') + 1

            if num_lines >= min_lines:
                processed_sentences.append(" ".join(current_block))
                current_block = []
                num_lines = 0

        # Add the last block if it's not empty
        if current_block:
            processed_sentences.append(" ".join(current_block))

        return processed_sentences

    def process_text(self, note_id, text, annotations):
        sentences = self.split_into_sentences(text, self.min_lines)
        start_index = 0
        for i, sentence in enumerate(sentences[:self.k]):
            sentence_start = text.find(sentence)
            sentence_end = sentence_start + len(sentence)
            entities = [(text[s:e], s, e, cid) for s, e, cid in annotations if s >= sentence_start and e <= sentence_end]
            masked_sentence, structure = self.mask_text(sentence, entities, sentence_start, sentence_end)
            regenerated_sentence = self.regenerate_text(masked_sentence)

            # Replace <extra_id_x> with <mask> for printing
            display_masked_sentence = re.sub(r'<extra_id_\d+>', '<mask>', masked_sentence)
            print(f"\n\033[1mNote ID: {note_id}, Sentence {i+1}\033[0m")
            print("\n\033[1mOriginal Sentence:\033[0m\n", sentence)
            print("\n\033[1mEntities:\033[0m\n", entities)
            print("\n\033[1mStructure:\033[0m\n", structure)
            print("\n\033[1mMasked Sentence:\033[0m\n", display_masked_sentence)
            print("\n\033[1mGenerated Text:\033[0m\n", regenerated_sentence)

            # # Compute ROUGE-L score
            # rouge_scores = self.scorer.score(sentence, regenerated_sentence)['rougeL']
            # print("\n\033[1mROUGE-L Score:\033[0m")
            # print(f"Precision: {rouge_scores.precision:.4f}, Recall: {rouge_scores.recall:.4f}, F1: {rouge_scores.fmeasure:.4f}")

            # # Compute BERTScore using 'bert-base-uncased' model
            # P, R, F1 = bert_score_fn([regenerated_sentence], [sentence], lang="en", model_type="bert-base-uncased", rescale_with_baseline=True)
            # # P, R, F1 = bert_score_fn([regenerated_sentence], [sentence], lang="en", model_type="bert-base-uncased", rescale_with_baseline=True, show_progress_bar=False)
            # print("\n\033[1mBERTScore:\033[0m")
            # print(f"Precision: {P.mean().item():.4f}, Recall: {R.mean().item():.4f}, F1: {F1.mean().item():.4f}")
            rouge_results = self.rouge.compute(predictions=[regenerated_sentence], references=[sentence])
            P, R, F1 = bert_score_fn([regenerated_sentence], [sentence], lang="en", rescale_with_baseline=True)

            rouge_1 = rouge_results['rouge1'] * 100
            rouge_2 = rouge_results['rouge2'] * 100
            rouge_L = rouge_results['rougeL'] * 100
            bertscore_f1 = F1.mean().cpu().item()
            flesch_reading_ease = textstat.flesch_reading_ease(regenerated_sentence)

            # METEOR score
            tokenized_sentence = word_tokenize(sentence)
            tokenized_regeneration = word_tokenize(regenerated_sentence)
            meteor = meteor_score([tokenized_sentence], tokenized_regeneration)

            flesch_reading_ease_ori = textstat.flesch_reading_ease(sentence)
            flesch_reading_ease_mask = textstat.flesch_reading_ease(masked_sentence)
            rouge_results_mask = self.rouge.compute(predictions=[masked_sentence], references=[sentence])
            rouge_1_mask = rouge_results_mask['rouge1'] * 100
            rouge_2_mask = rouge_results_mask['rouge2'] * 100
            rouge_L_mask = rouge_results_mask['rougeL'] * 100

            P_Mask, R_Mask, F1_Mask = bert_score_fn([masked_sentence], [sentence], lang="en", rescale_with_baseline=True)
            bertscore_f1_mask = F1_Mask.mean().cpu().item()

            # METEOR score
            tokenized_masked_sentence = word_tokenize(masked_sentence)
            meteor_mask = meteor_score([tokenized_sentence], tokenized_masked_sentence)

            print(f"\n\033[1mMetrics for Note ID: {note_id}, Sentence {i+1}\033[0m:")
            print("{:<25} {:<15} {:<15}".format("Metric", "Evaluation", "Baseline"))
            print("-" * 80)
            print("{:<25} {:<15.2f} {:<15.2f}".format("ROUGE-1", rouge_1, rouge_1_mask))
            print("{:<25} {:<15.2f} {:<15.2f}".format("ROUGE-2", rouge_2, rouge_2_mask))
            print("{:<25} {:<15.2f} {:<15.2f}".format("ROUGE-L", rouge_L, rouge_L_mask))
            print("{:<25} {:<15.2f} {:<15.2f}".format("BERTScore F1", bertscore_f1, bertscore_f1_mask))
            print("{:<25} {:<15.2f} {:<15.2f}".format("METEOR", meteor, meteor_mask))
            print("{:<25} {:<15.2f} {:<15}".format("Flesch Reading Ease", flesch_reading_ease, f"Ori: {flesch_reading_ease_ori:.2f}, Mask: {flesch_reading_ease_mask:.2f}"))


            print("="*80)

    def mask_text(self, sentence, entities, sentence_start, sentence_end):
        tokens = word_tokenize(sentence)
        original_tokens = tokens[:]  # Copy of original tokens
        mask = [True] * len(tokens)  # Initialize mask list, assuming all tokens can be masked

        # Generate character offsets for each token using word_tokenize positions
        token_offsets = []
        current_position = 0
        for token in tokens:
            start_position = sentence.find(token, current_position)
            end_position = start_position + len(token)
            token_offsets.append((start_position, end_position))
            current_position = end_position

        structure_texts = []
        allowed_lower = {'a', 'the', 'of', 'in', 'on', 'at', 'or', 'by', 'for', 'with', 'about', 'as', 'an', 'to'}
        termination_punctuation = {'.', '_', '___', '!', '?', ':'}
        # non_termination_pattern = re.compile(r'[^a-zA-Z\.!_?___]')
        non_termination_pattern = re.compile(r'[^.!:_?___]')

        i = 0
        while i < len(tokens):
            if tokens[i].endswith(':'):
                # Start checking backwards from the colon
                title_start = i - 1
                while title_start >= 0:
                    current_token = tokens[title_start]
                    if current_token in termination_punctuation or (current_token.islower() and current_token not in allowed_lower):
                        break  # Stop if a termination punctuation or a non-allowed lowercase word is encountered
                    title_start -= 1
                title_start += 1  # Adjust to include the first valid word after the break point

                # Capture the title segment if valid
                if title_start < i:
                    title_segment = tokens[title_start:i + 1]  # Include the colon in the title
                    structure_text = ' '.join(title_segment)
                    if not structure_texts or (structure_texts and structure_text not in structure_texts[-1]):
                        structure_texts.append(structure_text)
                        # Mask the title segment
                        for j in range(title_start, i + 1):
                            mask[j] = False
                i = i + 1  # Ensure to move past the last processed colon
            else:
                i += 1

        # Protect entities based on their positions and non-alphabetic tokens
        special_pattern = re.compile(r'\w\s*[^\w\s]\s*\d')
        for idx, token in enumerate(tokens):
            if not token.isalpha() or token.isupper() or re.search(r'\d', token) or special_pattern.search(token):  # Check if the token is purely alphabetic
                mask[idx] = False

        # only mask entities (identify borders accuratly)
        for _, ent_start, ent_end, _ in entities:
            for idx, (token_start, token_end) in enumerate(token_offsets):
                if (token_start + sentence_start >= ent_start and token_start + sentence_start < ent_end) or \
                  (token_end + sentence_start > ent_start and token_end + sentence_start <= ent_end) or \
                  (token_start + sentence_start <= ent_start and token_end + sentence_start >= ent_end):
                    mask[idx] = False

        for match in re.finditer(r'\w\s*[^\w\s]\s*\d', sentence):
            match_start, match_end = match.span()
            for idx, (token_start, token_end) in enumerate(token_offsets):
                if token_start >= match_start and token_end <= match_end:
                    mask[idx] = False
                    break

        # Calculate the indices of maskable tokens
        maskable_indices = [idx for idx, m in enumerate(mask) if m]
        if self.masking_ratio == 1.0:
            # If mask_ratio is 1.0, mask all maskable tokens
            mask_indices = maskable_indices
        else:
            num_to_mask = int(len(maskable_indices) * self.masking_ratio)
            mask_indices = random.sample(maskable_indices, num_to_mask) if maskable_indices else []

        mask_id = 0  # Initialize mask ID for T5 special tokens
        # Apply masks using T5 special tokens
        for idx in mask_indices:
            tokens[idx] = f'<extra_id_{mask_id}>'
            mask_id += 1

        # Reconstruct the sentence with original spacing preserved
        masked_sentence = ''
        last_end = 0
        for i, offset in enumerate(token_offsets):
            start, end = offset
            masked_sentence += sentence[last_end:start] + tokens[i]
            last_end = end
        masked_sentence += sentence[last_end:]  # Add any trailing part of the sentence

        structure = '    '.join(structure_texts)

        return masked_sentence, structure

    def regenerate_text(self, masked_sentence):
        # Prepare the input data for the model
        input_text = "In clinical background, fill in the blank: " + masked_sentence
        inputs = self.tokenizer(input_text, return_tensors="pt").to(self.device)

        with torch.no_grad():
            outputs = self.model.generate(
                inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                max_length=512,
                num_beams=5,
                no_repeat_ngram_size=2,
                early_stopping=True
            )
            regenerated_sentence = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        masked_tokens = masked_sentence.split()
        predicted_tokens = regenerated_sentence.split()

        final_tokens = []
        pred_idx = 0  # Index for predicted tokens

        mask_token_pattern = re.compile(r'<extra_id_\d+>')

        for token in masked_tokens:
            if mask_token_pattern.match(token):
                self.total_masks += 1  # Count the total number of masks
                if pred_idx < len(predicted_tokens):
                    predicted_token = predicted_tokens[pred_idx]
                    final_tokens.append(predicted_token)
                    # Check if the predicted token is valid (alphanumeric and not in the dictionary)
                    if not re.match(r'^[a-zA-Z0-9-]+$', predicted_token):
                        self.invalid_masks += 1
                    pred_idx += 1
                else:
                    # Append the original token if no predicted token is available
                    final_tokens.append(token)
                    self.invalid_masks += 1
            else:
                # Append the original token if it is not a mask token
                final_tokens.append(token)

        final_output = ' '.join(final_tokens)
        return final_output


    def display_sentences_with_entities(self):
        unique_ids = self.merged_df['note_id'].drop_duplicates().sample(n=self.small_size, random_state=self.random_seed)
        for note_id in unique_ids:
            group = self.merged_df[self.merged_df['note_id'] == note_id]
            text = group['text'].iloc[0]
            annotations = group[['start', 'end', 'concept_id']].dropna().astype(int).to_numpy()
            self.process_text(note_id, text, annotations)

        # Calculate and display the invalid prediction mask ratio
        invalid_ratio = self.invalid_masks / self.total_masks if self.total_masks > 0 else 0
        # print("\n\033[1mInvalid Prediction Mask Ratio: \033[0m\n", invalid_ratio)
        print(f"\n\033[1mInvalid Prediction Mask Ratio: {invalid_ratio:.2f}\033[0m\n")


augmentor = MLMAugmentor(
    notes_path='/content/drive/MyDrive/MSc_Project/dataset/MimicIV/snomed_ct_entity_linking_challenge_1.0.0/snomed_ct_entity_linking_challenge_1.0.0/mimicIv_notes_training_set.csv',
    annotations_path='/content/drive/MyDrive/MSc_Project/dataset/MimicIV/snomed_ct_entity_linking_challenge_1.0.0/snomed_ct_entity_linking_challenge_1.0.0/train_annotations.csv',
    small_size=1,
    random_seed=42,
    masking_ratio=0.5,
    k=10
)
augmentor.display_sentences_with_entities()



Note ID: 10807423-DS-19, Sentence 1

Original Sentence:
  
Name:  ___               Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   M
 
Service: ORTHOPAEDICS
 
Allergies: 
No Known Allergies / Adverse Drug Reactions
 
Attending: ___.

Entities:
 [('No Known Allergies', 181, 199, 609328004), ('Adverse Drug Reactions', 202, 224, 419511003)]

Structure:
 Name :    Unit No :    Admission Date :    Discharge Date :    Date of Birth :    Sex :    M Service :    ORTHOPAEDICS Allergies :    No Known Allergies / Adverse Drug Reactions Attending :

Masked Sentence:
  
Name:  ___               Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   M
 
Service: ORTHOPAEDICS
 
Allergies: 
No Known Allergies / Adverse Drug Reactions
 
Attending: ___.

Generated Text:
 Name: ___ Unit No: ___ Admission Date: ___ Discharge Date: ___ Date of Birth: ___ Sex: M Service: ORTHOPAEDIC

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Metrics for Note ID: 10807423-DS-19, Sentence 1:
Metric                    Evaluation      Baseline       
--------------------------------------------------------------------------------
ROUGE-1                   100.00          100.00         
ROUGE-2                   100.00          100.00         
ROUGE-L                   100.00          100.00         
BERTScore F1              0.12            1.00           
METEOR                    1.00            1.00           
Flesch Reading Ease       51.52           Ori: 51.52, Mask: 51.52

Note ID: 10807423-DS-19, Sentence 2

Original Sentence:
 Chief Complaint:
R ankle fracture dislocation, open
 
Major Surgical or Invasive Procedure:
ORIF R ankle and I&D ___

 
History of Present Illness:
Chief Complaint:  ankle pain
Reason for Orthopedics Consult: management of open fracture
 
HISTORY OF PRESENT ILLNESS: 
Patient is a ___ yo male previously healhty presenting w/ fall 
from 6 feet, from ladder.

Entities:
 [('R ankle', 262, 269, 6685

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Metrics for Note ID: 10807423-DS-19, Sentence 2:
Metric                    Evaluation      Baseline       
--------------------------------------------------------------------------------
ROUGE-1                   80.36           73.77          
ROUGE-2                   69.09           63.33          
ROUGE-L                   80.36           73.77          
BERTScore F1              0.40            0.50           
METEOR                    0.82            0.85           
Flesch Reading Ease       8.21            Ori: 8.21, Mask: 8.21

Note ID: 10807423-DS-19, Sentence 3

Original Sentence:
 Patient landed on LLE w/ forced 
eversion and subsequent open fracture/dislocation. Denies head 
strike or LOC.

Entities:
 [('LLE', 621, 624, 32153003), ('eversion', 636, 644, 4196002), ('open fracture', 660, 673, 397181002), ('dislocation', 674, 685, 87642003), ('head \nstrike', 694, 706, 82271004), ('LOC', 710, 713, 419045004)]

Structure:
 

Masked Sentence:
 <mask> <mask> on LLE w/ <mask> 
e

...


Metrics for Note ID: 10807423-DS-19, Sentence 8:
Metric                    Evaluation      Baseline       
--------------------------------------------------------------------------------
ROUGE-1                   90.91           77.78          
ROUGE-2                   77.42           64.71          
ROUGE-L                   90.91           77.78          
BERTScore F1              0.45            0.60           
METEOR                    0.94            0.90           
Flesch Reading Ease       21.06           Ori: 29.52, Mask: 21.06

Note ID: 10807423-DS-19, Sentence 9

Original Sentence:
 PAST MEDICAL HISTORY:  
none
 
MEDICATIONS:  
none
 
ALLERGIES:  
NKDA
 
SOCIAL HISTORY:  
Denies alcohol, drugs, smoking
 
PHYSICAL EXAM:  
GENERAL: Alert, oriented, no acute distress  
HEENT: Sclera anicteric, MMM, oropharynx clear  
NECK: C-spine is non-tender to palpation
LUNGS: Clear to auscultation bilaterally
CV: Regular rate and rhythm, 
ABD: soft, non-tender, non-distended, 
PELVIS: st

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Metrics for Note ID: 10807423-DS-19, Sentence 9:
Metric                    Evaluation      Baseline       
--------------------------------------------------------------------------------
ROUGE-1                   89.71           80.82          
ROUGE-2                   79.10           73.61          
ROUGE-L                   86.76           80.82          
BERTScore F1              0.35            0.66           
METEOR                    0.92            0.90           
Flesch Reading Ease       -15.83          Ori: -15.83, Mask: -15.83

Note ID: 10807423-DS-19, Sentence 10

Original Sentence:
 +DP. Unable to assess. Warm, well perfused, 2+ pulses, 
no clubbing, cyanosis or edema.

Entities:
 []

Structure:
 

Masked Sentence:
 +DP. <mask> <mask> assess. Warm, <mask> <mask>, 2+ pulses, 
no clubbing, <mask> <mask> edema.

Generated Text:
 +DP. [State] cyanosis assess. Warm, or[MO] +[Reg#]. 2+ pulses, no clubbing, unable to edema.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Metrics for Note ID: 10807423-DS-19, Sentence 10:
Metric                    Evaluation      Baseline       
--------------------------------------------------------------------------------
ROUGE-1                   82.76           40.00          
ROUGE-2                   37.04           21.05          
ROUGE-L                   55.17           40.00          
BERTScore F1              0.31            0.02           
METEOR                    0.73            0.67           
Flesch Reading Ease       83.62           Ori: 81.29, Mask: 55.91

Invalid Prediction Mask Ratio: 0.59



`Clinical-T5 replace all DEID tags with special tokens, and these tokens are then added to the vocabulary of the models.
initialized from T5-Base, using Mimic-Ⅲ and Mimic-IV, trained on several downstream tasks, including SQUAD(Stanford Question Answering Dataset).

Clinical-T5-Scratch: use T5-Base, but randomly initialize the weights.

Clinical-T5-Sci: initialized from SciFive

In [ ]:
# Path test

import os
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/MSc_Project/Models/Clinical_T5_Sci'
if os.path.exists(path):
    print("Path exists. Here are the files:")
    print(os.listdir(path))
else:
    print("Path does not exist.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Path exists. Here are the files:
['tokenizer_config.json', 'special_tokens_map.json', 'config.json', 'added_tokens.json', 'spiece.model', 'tokenizer.json', 'pytorch_model.bin']


In [ ]:
!pip install rouge-score
!pip install bert-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=185b70f9d1f4d866131aa4a32e6c409d69c2c079ea796a8ce9adbb60e719fc9e
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinu

In [ ]:
import re
import random
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
from rouge_score import rouge_scorer
import bert_score
from bert_score import score as bert_score_fn
import logging

nltk.download('punkt')  # for sentence tokenization
from google.colab import drive
drive.mount('/content/drive')

logging.getLogger().setLevel(logging.WARNING)
logging.getLogger("bert_score").setLevel(logging.ERROR)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class MLMAugmentor:
    def __init__(self, notes_path, annotations_path, small_size=10, random_seed=42, masking_ratio=0.3, k=3):
        self.notes_path = notes_path
        self.annotations_path = annotations_path
        self.small_size = small_size
        self.random_seed = random_seed
        self.masking_ratio = masking_ratio
        self.k = k
        self.invalid_masks = 0
        self.total_masks = 0
        random.seed(self.random_seed)  # for reproducibility
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/MSc_Project/Models/Clinical_T5_Sci')
        self.model = AutoModelForSeq2SeqLM.from_pretrained('/content/drive/MyDrive/MSc_Project/Models/Clinical_T5_Sci').to(self.device)
        self.scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
        self.load_and_merge_datasets()

    def load_and_merge_datasets(self):
        try:
            notes_df = pd.read_csv(self.notes_path)
            annotations_df = pd.read_csv(self.annotations_path)
            self.merged_df = pd.merge(notes_df, annotations_df, on='note_id', how='left')
        except Exception as e:
            print(f"An error occurred: {e}")

    def process_text(self, note_id, text, annotations):
        sentences = sent_tokenize(text)
        for i, sentence in enumerate(sentences[:self.k]):
            sentence_start = text.find(sentence)
            sentence_end = sentence_start + len(sentence)
            entities = [(text[s:e], s, e, cid) for s, e, cid in annotations if s >= sentence_start and e <= sentence_end]
            masked_sentence, structure = self.mask_text(sentence, entities, sentence_start, sentence_end)
            regenerated_sentence = self.regenerate_text(masked_sentence)

            # Replace <extra_id_x> with <mask> for printing
            display_masked_sentence = re.sub(r'<extra_id_\d+>', '<mask>', masked_sentence)
            print(f"\n\033[1mNote ID: {note_id}, Sentence {i+1}\033[0m")
            print("\n\033[1mOriginal Sentence:\033[0m\n", sentence)
            print("\n\033[1mEntities:\033[0m\n", entities)
            print("\n\033[1mStructure:\033[0m\n", structure)
            print("\n\033[1mMasked Sentence:\033[0m\n", display_masked_sentence)
            print("\n\033[1mGenerated Text:\033[0m\n", regenerated_sentence)

            # Compute ROUGE-L score
            rouge_scores = self.scorer.score(sentence, regenerated_sentence)['rougeL']
            print("\n\033[1mROUGE-L Score:\033[0m")
            print(f"Precision: {rouge_scores.precision:.4f}, Recall: {rouge_scores.recall:.4f}, F1: {rouge_scores.fmeasure:.4f}")

            # Compute BERTScore using 'bert-base-uncased' model
            P, R, F1 = bert_score_fn([regenerated_sentence], [sentence], lang="en", model_type="bert-base-uncased", rescale_with_baseline=True)
            # P, R, F1 = bert_score_fn([regenerated_sentence], [sentence], lang="en", model_type="bert-base-uncased", rescale_with_baseline=True, show_progress_bar=False)
            print("\n\033[1mBERTScore:\033[0m")
            print(f"Precision: {P.mean().item():.4f}, Recall: {R.mean().item():.4f}, F1: {F1.mean().item():.4f}")

            print("="*80)

    def mask_text(self, sentence, entities, sentence_start, sentence_end):
        tokens = word_tokenize(sentence)
        original_tokens = tokens[:]  # Copy of original tokens
        mask = [True] * len(tokens)  # Initialize mask list, assuming all tokens can be masked

        # Generate character offsets for each token using word_tokenize positions
        token_offsets = []
        current_position = 0
        for token in tokens:
            start_position = sentence.find(token, current_position)
            end_position = start_position + len(token)
            token_offsets.append((start_position, end_position))
            current_position = end_position

        structure_texts = []
        allowed_lower = {'a', 'the', 'of', 'in', 'on', 'at', 'or', 'by', 'for', 'with', 'about', 'as', 'an', 'to'}
        termination_punctuation = {'.', '_', '___', '!', '?', ':'}
        # non_termination_pattern = re.compile(r'[^a-zA-Z\.!_?___]')
        non_termination_pattern = re.compile(r'[^.!:_?___]')

        i = 0
        while i < len(tokens):
            if tokens[i].endswith(':'):
                # Start checking backwards from the colon
                title_start = i - 1
                while title_start >= 0:
                    current_token = tokens[title_start]
                    if current_token in termination_punctuation or (current_token.islower() and current_token not in allowed_lower):
                        break  # Stop if a termination punctuation or a non-allowed lowercase word is encountered
                    title_start -= 1
                title_start += 1  # Adjust to include the first valid word after the break point

                # Capture the title segment if valid
                if title_start < i:
                    title_segment = tokens[title_start:i + 1]  # Include the colon in the title
                    structure_text = ' '.join(title_segment)
                    if not structure_texts or (structure_texts and structure_text not in structure_texts[-1]):
                        structure_texts.append(structure_text)
                        # Mask the title segment
                        for j in range(title_start, i + 1):
                            mask[j] = False
                i = i + 1  # Ensure to move past the last processed colon
            else:
                i += 1

        # Protect entities based on their positions and non-alphabetic tokens
        special_pattern = re.compile(r'\w\s*[^\w\s]\s*\d')
        for idx, token in enumerate(tokens):
            if not token.isalpha() or token.isupper() or re.search(r'\d', token) or special_pattern.search(token):  # Check if the token is purely alphabetic
                mask[idx] = False

        # only mask entities (identify borders accuratly)
        for _, ent_start, ent_end, _ in entities:
            for idx, (token_start, token_end) in enumerate(token_offsets):
                if (token_start + sentence_start >= ent_start and token_start + sentence_start < ent_end) or \
                  (token_end + sentence_start > ent_start and token_end + sentence_start <= ent_end) or \
                  (token_start + sentence_start <= ent_start and token_end + sentence_start >= ent_end):
                    mask[idx] = False

        for match in re.finditer(r'\w\s*[^\w\s]\s*\d', sentence):
            match_start, match_end = match.span()
            for idx, (token_start, token_end) in enumerate(token_offsets):
                if token_start >= match_start and token_end <= match_end:
                    mask[idx] = False
                    break

        # Calculate the indices of maskable tokens
        maskable_indices = [idx for idx, m in enumerate(mask) if m]
        if self.masking_ratio == 1.0:
            # If mask_ratio is 1.0, mask all maskable tokens
            mask_indices = maskable_indices
        else:
            num_to_mask = int(len(maskable_indices) * self.masking_ratio)
            mask_indices = random.sample(maskable_indices, num_to_mask) if maskable_indices else []

        mask_id = 0  # Initialize mask ID for T5 special tokens
        # Apply masks using T5 special tokens
        for idx in mask_indices:
            tokens[idx] = f'<extra_id_{mask_id}>'
            mask_id += 1

        # Reconstruct the sentence with original spacing preserved
        masked_sentence = ''
        last_end = 0
        for i, offset in enumerate(token_offsets):
            start, end = offset
            masked_sentence += sentence[last_end:start] + tokens[i]
            last_end = end
        masked_sentence += sentence[last_end:]  # Add any trailing part of the sentence

        structure = '    '.join(structure_texts)

        return masked_sentence, structure

    def regenerate_text(self, masked_sentence):
        # Prepare the input data for the model
        input_text = "In clinical background, fill in the blank: " + masked_sentence
        inputs = self.tokenizer(input_text, return_tensors="pt").to(self.device)

        with torch.no_grad():
            outputs = self.model.generate(
                inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                max_length=512,
                num_beams=5,
                no_repeat_ngram_size=2,
                early_stopping=True
            )
            regenerated_sentence = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        masked_tokens = masked_sentence.split()
        predicted_tokens = regenerated_sentence.split()

        final_tokens = []
        pred_idx = 0  # Index for predicted tokens

        mask_token_pattern = re.compile(r'<extra_id_\d+>')

        for token in masked_tokens:
            if mask_token_pattern.match(token):
                self.total_masks += 1  # Count the total number of masks
                if pred_idx < len(predicted_tokens):
                    predicted_token = predicted_tokens[pred_idx]
                    final_tokens.append(predicted_token)
                    # Check if the predicted token is valid (alphanumeric and not in the dictionary)
                    if not re.match(r'^[a-zA-Z0-9-]+$', predicted_token):
                        self.invalid_masks += 1
                    pred_idx += 1
                else:
                    # Append the original token if no predicted token is available
                    final_tokens.append(token)
                    self.invalid_masks += 1
            else:
                # Append the original token if it is not a mask token
                final_tokens.append(token)

        final_output = ' '.join(final_tokens)
        return final_output


    def display_sentences_with_entities(self):
        unique_ids = self.merged_df['note_id'].drop_duplicates().sample(n=self.small_size, random_state=self.random_seed)
        for note_id in unique_ids:
            group = self.merged_df[self.merged_df['note_id'] == note_id]
            text = group['text'].iloc[0]
            annotations = group[['start', 'end', 'concept_id']].dropna().astype(int).to_numpy()
            self.process_text(note_id, text, annotations)

        # Calculate and display the invalid prediction mask ratio
        invalid_ratio = self.invalid_masks / self.total_masks if self.total_masks > 0 else 0
        # print("\n\033[1mInvalid Prediction Mask Ratio: \033[0m\n", invalid_ratio)
        print(f"\n\033[1mInvalid Prediction Mask Ratio: {invalid_ratio:.2f}\033[0m\n")


augmentor = MLMAugmentor(
    notes_path='/content/drive/MyDrive/MSc_Project/dataset/MimicIV/snomed_ct_entity_linking_challenge_1.0.0/snomed_ct_entity_linking_challenge_1.0.0/mimicIv_notes_training_set.csv',
    annotations_path='/content/drive/MyDrive/MSc_Project/dataset/MimicIV/snomed_ct_entity_linking_challenge_1.0.0/snomed_ct_entity_linking_challenge_1.0.0/train_annotations.csv',
    small_size=5,
    random_seed=42,
    masking_ratio=0.5,
    k=30
)
augmentor.display_sentences_with_entities()


KeyboardInterrupt: 

split in blocks

In [ ]:
class MLMAugmentor:
    def __init__(self, notes_path, annotations_path, small_size=10, random_seed=42, masking_ratio=0.3, k=3, min_lines=3):
        self.notes_path = notes_path
        self.annotations_path = annotations_path
        self.small_size = small_size
        self.random_seed = random_seed
        self.masking_ratio = masking_ratio
        self.k = k
        self.min_lines = min_lines
        self.invalid_masks = 0
        self.total_masks = 0
        random.seed(self.random_seed)  # for reproducibility
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/MSc_Project/Models/Clinical_T5_Sci')
        self.model = AutoModelForSeq2SeqLM.from_pretrained('/content/drive/MyDrive/MSc_Project/Models/Clinical_T5_Sci').to(self.device)
        self.scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
        self.load_and_merge_datasets()

    def load_and_merge_datasets(self):
        try:
            notes_df = pd.read_csv(self.notes_path)
            annotations_df = pd.read_csv(self.annotations_path)
            self.merged_df = pd.merge(notes_df, annotations_df, on='note_id', how='left')
        except Exception as e:
            print(f"An error occurred: {e}")

    def split_into_sentences(self, text, min_lines=3):
        # Splitting text into sentence
        sentences = sent_tokenize(text)
        processed_sentences = []
        current_block = []
        num_lines = 0


        for sentence in sentences:
            current_block.append(sentence)
            num_lines += sentence.count('\n') + 1

            if num_lines >= min_lines:
                processed_sentences.append(" ".join(current_block))
                current_block = []
                num_lines = 0

        # Add the last block if it's not empty
        if current_block:
            processed_sentences.append(" ".join(current_block))

        return processed_sentences

    def process_text(self, note_id, text, annotations):
        sentences = self.split_into_sentences(text, self.min_lines)
        start_index = 0
        for i, sentence in enumerate(sentences[:self.k]):
            sentence_start = text.find(sentence)
            sentence_end = sentence_start + len(sentence)
            entities = [(text[s:e], s, e, cid) for s, e, cid in annotations if s >= sentence_start and e <= sentence_end]
            masked_sentence, structure = self.mask_text(sentence, entities, sentence_start, sentence_end)
            regenerated_sentence = self.regenerate_text(masked_sentence)

            # Replace <extra_id_x> with <mask> for printing
            display_masked_sentence = re.sub(r'<extra_id_\d+>', '<mask>', masked_sentence)
            print(f"\n\033[1mNote ID: {note_id}, Sentence {i+1}\033[0m")
            print("\n\033[1mOriginal Sentence:\033[0m\n", sentence)
            print("\n\033[1mEntities:\033[0m\n", entities)
            print("\n\033[1mStructure:\033[0m\n", structure)
            print("\n\033[1mMasked Sentence:\033[0m\n", display_masked_sentence)
            print("\n\033[1mGenerated Text:\033[0m\n", regenerated_sentence)

            # Compute ROUGE-L score
            rouge_scores = self.scorer.score(sentence, regenerated_sentence)['rougeL']
            print("\n\033[1mROUGE-L Score:\033[0m")
            print(f"Precision: {rouge_scores.precision:.4f}, Recall: {rouge_scores.recall:.4f}, F1: {rouge_scores.fmeasure:.4f}")

            # Compute BERTScore using 'bert-base-uncased' model
            P, R, F1 = bert_score_fn([regenerated_sentence], [sentence], lang="en", model_type="bert-base-uncased", rescale_with_baseline=True)
            # P, R, F1 = bert_score_fn([regenerated_sentence], [sentence], lang="en", model_type="bert-base-uncased", rescale_with_baseline=True, show_progress_bar=False)
            print("\n\033[1mBERTScore:\033[0m")
            print(f"Precision: {P.mean().item():.4f}, Recall: {R.mean().item():.4f}, F1: {F1.mean().item():.4f}")

            print("="*80)

    def mask_text(self, sentence, entities, sentence_start, sentence_end):
        tokens = word_tokenize(sentence)
        original_tokens = tokens[:]  # Copy of original tokens
        mask = [True] * len(tokens)  # Initialize mask list, assuming all tokens can be masked

        # Generate character offsets for each token using word_tokenize positions
        token_offsets = []
        current_position = 0
        for token in tokens:
            start_position = sentence.find(token, current_position)
            end_position = start_position + len(token)
            token_offsets.append((start_position, end_position))
            current_position = end_position

        structure_texts = []
        allowed_lower = {'a', 'the', 'of', 'in', 'on', 'at', 'or', 'by', 'for', 'with', 'about', 'as', 'an', 'to'}
        termination_punctuation = {'.', '_', '___', '!', '?', ':'}
        # non_termination_pattern = re.compile(r'[^a-zA-Z\.!_?___]')
        non_termination_pattern = re.compile(r'[^.!:_?___]')

        i = 0
        while i < len(tokens):
            if tokens[i].endswith(':'):
                # Start checking backwards from the colon
                title_start = i - 1
                while title_start >= 0:
                    current_token = tokens[title_start]
                    if current_token in termination_punctuation or (current_token.islower() and current_token not in allowed_lower):
                        break  # Stop if a termination punctuation or a non-allowed lowercase word is encountered
                    title_start -= 1
                title_start += 1  # Adjust to include the first valid word after the break point

                # Capture the title segment if valid
                if title_start < i:
                    title_segment = tokens[title_start:i + 1]  # Include the colon in the title
                    structure_text = ' '.join(title_segment)
                    if not structure_texts or (structure_texts and structure_text not in structure_texts[-1]):
                        structure_texts.append(structure_text)
                        # Mask the title segment
                        for j in range(title_start, i + 1):
                            mask[j] = False
                i = i + 1  # Ensure to move past the last processed colon
            else:
                i += 1

        # Protect entities based on their positions and non-alphabetic tokens
        special_pattern = re.compile(r'\w\s*[^\w\s]\s*\d')
        for idx, token in enumerate(tokens):
            if not token.isalpha() or token.isupper() or re.search(r'\d', token) or special_pattern.search(token):  # Check if the token is purely alphabetic
                mask[idx] = False

        # only mask entities (identify borders accuratly)
        for _, ent_start, ent_end, _ in entities:
            for idx, (token_start, token_end) in enumerate(token_offsets):
                if (token_start + sentence_start >= ent_start and token_start + sentence_start < ent_end) or \
                  (token_end + sentence_start > ent_start and token_end + sentence_start <= ent_end) or \
                  (token_start + sentence_start <= ent_start and token_end + sentence_start >= ent_end):
                    mask[idx] = False

        for match in re.finditer(r'\w\s*[^\w\s]\s*\d', sentence):
            match_start, match_end = match.span()
            for idx, (token_start, token_end) in enumerate(token_offsets):
                if token_start >= match_start and token_end <= match_end:
                    mask[idx] = False
                    break

        # Calculate the indices of maskable tokens
        maskable_indices = [idx for idx, m in enumerate(mask) if m]
        if self.masking_ratio == 1.0:
            # If mask_ratio is 1.0, mask all maskable tokens
            mask_indices = maskable_indices
        else:
            num_to_mask = int(len(maskable_indices) * self.masking_ratio)
            mask_indices = random.sample(maskable_indices, num_to_mask) if maskable_indices else []

        mask_id = 0  # Initialize mask ID for T5 special tokens
        # Apply masks using T5 special tokens
        for idx in mask_indices:
            tokens[idx] = f'<extra_id_{mask_id}>'
            mask_id += 1

        # Reconstruct the sentence with original spacing preserved
        masked_sentence = ''
        last_end = 0
        for i, offset in enumerate(token_offsets):
            start, end = offset
            masked_sentence += sentence[last_end:start] + tokens[i]
            last_end = end
        masked_sentence += sentence[last_end:]  # Add any trailing part of the sentence

        structure = '    '.join(structure_texts)

        return masked_sentence, structure

    def regenerate_text(self, masked_sentence):
        # Prepare the input data for the model
        input_text = "In clinical background, fill in the blank: " + masked_sentence
        inputs = self.tokenizer(input_text, return_tensors="pt").to(self.device)

        with torch.no_grad():
            outputs = self.model.generate(
                inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                max_length=512,
                num_beams=5,
                no_repeat_ngram_size=2,
                early_stopping=True
            )
            regenerated_sentence = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        masked_tokens = masked_sentence.split()
        predicted_tokens = regenerated_sentence.split()

        final_tokens = []
        pred_idx = 0  # Index for predicted tokens

        mask_token_pattern = re.compile(r'<extra_id_\d+>')

        for token in masked_tokens:
            if mask_token_pattern.match(token):
                self.total_masks += 1  # Count the total number of masks
                if pred_idx < len(predicted_tokens):
                    predicted_token = predicted_tokens[pred_idx]
                    final_tokens.append(predicted_token)
                    # Check if the predicted token is valid (alphanumeric and not in the dictionary)
                    if not re.match(r'^[a-zA-Z0-9-]+$', predicted_token):
                        self.invalid_masks += 1
                    pred_idx += 1
                else:
                    # Append the original token if no predicted token is available
                    final_tokens.append(token)
                    self.invalid_masks += 1
            else:
                # Append the original token if it is not a mask token
                final_tokens.append(token)

        final_output = ' '.join(final_tokens)
        return final_output


    def display_sentences_with_entities(self):
        unique_ids = self.merged_df['note_id'].drop_duplicates().sample(n=self.small_size, random_state=self.random_seed)
        for note_id in unique_ids:
            group = self.merged_df[self.merged_df['note_id'] == note_id]
            text = group['text'].iloc[0]
            annotations = group[['start', 'end', 'concept_id']].dropna().astype(int).to_numpy()
            self.process_text(note_id, text, annotations)

        # Calculate and display the invalid prediction mask ratio
        invalid_ratio = self.invalid_masks / self.total_masks if self.total_masks > 0 else 0
        # print("\n\033[1mInvalid Prediction Mask Ratio: \033[0m\n", invalid_ratio)
        print(f"\n\033[1mInvalid Prediction Mask Ratio: {invalid_ratio:.2f}\033[0m\n")


augmentor = MLMAugmentor(
    notes_path='/content/drive/MyDrive/MSc_Project/dataset/MimicIV/snomed_ct_entity_linking_challenge_1.0.0/snomed_ct_entity_linking_challenge_1.0.0/mimicIv_notes_training_set.csv',
    annotations_path='/content/drive/MyDrive/MSc_Project/dataset/MimicIV/snomed_ct_entity_linking_challenge_1.0.0/snomed_ct_entity_linking_challenge_1.0.0/train_annotations.csv',
    small_size=5,
    random_seed=42,
    masking_ratio=0.8,
    k=30
)
augmentor.display_sentences_with_entities()



Note ID: 10807423-DS-19, Sentence 1

Original Sentence:
  
Name:  ___               Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   M
 
Service: ORTHOPAEDICS
 
Allergies: 
No Known Allergies / Adverse Drug Reactions
 
Attending: ___.

Entities:
 [('No Known Allergies', 181, 199, 609328004), ('Adverse Drug Reactions', 202, 224, 419511003)]

Structure:
 Name :    Unit No :    Admission Date :    Discharge Date :    Date of Birth :    Sex :    M Service :    ORTHOPAEDICS Allergies :    No Known Allergies / Adverse Drug Reactions Attending :

Masked Sentence:
  
Name:  ___               Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   M
 
Service: ORTHOPAEDICS
 
Allergies: 
No Known Allergies / Adverse Drug Reactions
 
Attending: ___.

Generated Text:
 Name: ___ Unit No: ___ Admission Date: ___ Discharge Date: ___ Date of Birth: ___ Sex: M Service: ORTHOPAEDIC